## DSCI 235 Final Assignment

#### Possible datasets

Spotify hit predictor: https://www.kaggle.com/theoverman/the-spotify-hit-predictor-dataset

#### Questions to Answer

1) Is pop music formulaic?

2) What are the best predictors for what will become a hit song?

3) What is the 'ideal' pop song?

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt

%autosave 60

Autosaving every 60 seconds


In [36]:
sixties = pd.read_csv('https://raw.githubusercontent.com/ecreagar/DSCI235/master/the-spotify-hit-predictor-dataset/dataset-of-60s.csv')
seventies = pd.read_csv('https://raw.githubusercontent.com/ecreagar/DSCI235/master/the-spotify-hit-predictor-dataset/dataset-of-70s.csv')
eighties = pd.read_csv('https://raw.githubusercontent.com/ecreagar/DSCI235/master/the-spotify-hit-predictor-dataset/dataset-of-80s.csv')
nineties = pd.read_csv('https://raw.githubusercontent.com/ecreagar/DSCI235/master/the-spotify-hit-predictor-dataset/dataset-of-90s.csv')
thousands = pd.read_csv('https://raw.githubusercontent.com/ecreagar/DSCI235/master/the-spotify-hit-predictor-dataset/dataset-of-00s.csv')
tens = pd.read_csv('https://raw.githubusercontent.com/ecreagar/DSCI235/master/the-spotify-hit-predictor-dataset/dataset-of-10s.csv')

songs = pd.concat([sixties,seventies,eighties,nineties,thousands,tens], ignore_index=True, keys=['6', '7', '8', '9', '0', '1'])
songs

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Jealous Kind Of Fella,Garland Green,spotify:track:1dtKN6wwlolkM8XZy2y9C1,0.417,0.620,3,-7.727,1,0.0403,0.4900,0.000000,0.0779,0.8450,185.655,173533,3,32.94975,9,1
1,Initials B.B.,Serge Gainsbourg,spotify:track:5hjsmSnUefdUqzsDogisiX,0.498,0.505,3,-12.475,1,0.0337,0.0180,0.107000,0.1760,0.7970,101.801,213613,4,48.82510,10,0
2,Melody Twist,Lord Melody,spotify:track:6uk8tI6pwxxdVTNlNOJeJh,0.657,0.649,5,-13.392,1,0.0380,0.8460,0.000004,0.1190,0.9080,115.940,223960,4,37.22663,12,0
3,Mi Bomba Sonó,Celia Cruz,spotify:track:7aNjMJ05FvUXACPWZ7yJmv,0.590,0.545,7,-12.058,0,0.1040,0.7060,0.024600,0.0610,0.9670,105.592,157907,4,24.75484,8,0
4,Uravu Solla,P. Susheela,spotify:track:1rQ0clvgkzWr001POOPJWx,0.515,0.765,11,-3.515,0,0.1240,0.8570,0.000872,0.2130,0.9060,114.617,245600,4,21.79874,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41101,Lotus Flowers,Yolta,spotify:track:4t1TljQWJ6ZuoSY67zVvBI,0.172,0.358,9,-14.430,1,0.0342,0.8860,0.966000,0.3140,0.0361,72.272,150857,4,24.30824,7,0
41102,Calling My Spirit,Kodak Black,spotify:track:2MShy1GSSgbmGUxADNIao5,0.910,0.366,1,-9.954,1,0.0941,0.0996,0.000000,0.2610,0.7400,119.985,152000,4,32.53856,8,1
41103,Teenage Dream,Katy Perry,spotify:track:55qBw1900pZKfXJ6Q9A2Lc,0.719,0.804,10,-4.581,1,0.0355,0.0132,0.000003,0.1390,0.6050,119.999,227760,4,20.73371,7,1
41104,Stormy Weather,Oscar Peterson,spotify:track:4o9npmYHrOF1rUxxTVH8h4,0.600,0.177,7,-16.070,1,0.0561,0.9890,0.868000,0.1490,0.5600,120.030,213387,4,21.65301,14,0
